In [1]:
import pandas as pd
import torch
from tqdm import tqdm
from sacrebleu.metrics import BLEU
from bert_score import score
from googletrans import Translator
import time
import numpy as np


d:\Data\HCMUS\Y4\Y4_S1\Nhap_mon_xu_ly_ngon_ngu_tu_nhien\Mid_term\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Config
INPUT_FILE = '../output/JSON2_4653_5815.csv' 
SAMPLE_SIZE = 200  # Số lượng câu lấy mẫu để đánh giá 
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

In [3]:
def get_proxy_translations(df, sample_size):
    """
    Dịch câu nguồn tiếng Trung sang tiếng Việt dùng Google Translate
    để làm tham chiếu so sánh.
    """
    print(f"Đang lấy mẫu ngẫu nhiên {sample_size} cặp câu để đánh giá...")
    
    # Lấy mẫu ngẫu nhiên
    sample = df.sample(n=min(sample_size, len(df)), random_state=0).copy()

    translator = Translator()
    hypotheses = [] # Câu trong file 
    references = [] # Câu dịch máy (Làm chuẩn so sánh)
    success_count = 0
    
    for idx, row in tqdm(sample.iterrows(), total=len(sample)):
        src = str(row['src_lang'])
        tgt = str(row['tgt_lang']) # Đây là câu tiếng Việt gốc trong dataset
        
        # Bỏ qua câu quá ngắn hoặc rỗng
        if len(src) < 2 or len(tgt) < 5: 
            continue
            
        try:
            # Dịch Trung -> Việt
            translated = translator.translate(src, src='zh-cn', dest='vi').text
            
            # Câu Google dịch là Reference (Tham chiếu) và câu Tgt gốc là Hypothesis (Giả thuyết)
            references.append(translated) 
            hypotheses.append(tgt)
            
            success_count += 1
            time.sleep(0.5)
            
        except Exception as e:
            # Nếu lỗi mạng thì bỏ qua
            continue
            
    print(f"Đã dịch thành công {success_count} câu.")
    return hypotheses, references

def calculate_metrics(hypotheses, references):
    """Tính toán BLEU và BERTScore"""
    if not hypotheses:
        print("Không có dữ liệu để tính toán.")
        return

    # Tính BLEU Score
    # BLEU đánh giá độ khớp từ vựng (n-gram overlap)
    bleu = BLEU()
    bleu_score = bleu.corpus_score(hypotheses, [references])
    
    print(f"BLEU Score: {bleu_score.score:.2f}")

    # Tính BERTScore
    # BERTScore đánh giá độ tương đồng ngữ nghĩa (Semantic Similarity) dùng mô hình BERT
    P, R, F1 = score(hypotheses, references, lang='vi', verbose=False, device=DEVICE)
    
    avg_bert = F1.mean().item()
    print(f"BERTScore (F1): {avg_bert:.4f}")

    print("Ví dụ tham chiếu (Top 3):")
    for i in range(min(3, len(hypotheses))):
        print(f"Cặp {i+1}:")
        print(f"Dataset Tgt: {hypotheses[i]}")
        print(f"Google Trans: {references[i]}")
        print("---")

In [4]:
df = pd.read_csv(INPUT_FILE)
# Thực hiện quy trình đánh giá
hyps, refs = get_proxy_translations(df, SAMPLE_SIZE)
# Tính điểm
calculate_metrics(hyps, refs)

Đang lấy mẫu ngẫu nhiên 200 cặp câu để đánh giá...


100%|██████████| 200/200 [07:08<00:00,  2.14s/it]


Đã dịch thành công 200 câu.
BLEU Score: 45.96
BERTScore (F1): 0.9092
Ví dụ tham chiếu (Top 3):
Cặp 1:
Dataset Tgt: Công cụ tương quan, đột biến nào tương quan với phản ứng thuốc nào, những thay đổi về lượng protein nào, tương quan với những thay đổi nào trong phản ứng thuốc.
Google Trans: Các công cụ liên quan, đột biến nào có liên quan đến phản ứng thuốc nào, sự thay đổi về lượng protein nào có liên quan đến thay đổi phản ứng thuốc nào.
---
Cặp 2:
Dataset Tgt: Ví dụ, có một đứa trẻ có nhu cầu đặc biệt, cha mẹ gặp khó khăn trong việc thoát khỏi vai trò người chăm sóc.
Google Trans: Ví dụ, với một đứa trẻ có nhu cầu đặc biệt, cha mẹ có thể khó rời khỏi vai trò người chăm sóc.
---
Cặp 3:
Dataset Tgt: Các hang động cuối cùng sẽ đi về bên phải nhưng có một nhánh nơi khối u thực sự đi vào tĩnh mạch chủ dưới, vì vậy những bệnh nhân này phục hồi rất tốt, đó là một khối u nang, và tất nhiên
Google Trans: Cuối cùng, khoang này mở rộng sang bên phải, nhưng có một nhánh, và khối u thực sự đi vào 